In [1]:
import pandas as pd
import numpy as np
from time import time
import warnings
warnings.filterwarnings("ignore")

In [2]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

raw_data = pd.read_csv("./kddcup.data_10_percent_corrected", header=None, names = col_names)
raw_data['label'] = raw_data['label'].replace(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.', 'perl.', 'phf.', 'pod.', 'portsweep.', 'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'], 'attack')
data = raw_data

In [3]:
def categorical_converter(data):
    '''manually encode the three categorical columns into ordinals'''
    cols=['protocol_type','service','flag','label']
    for i in cols:
        data[i] = le.fit_transform(data[i])
    data = hot.fit_transform(data.loc[:,cols])

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
hot = OneHotEncoder(sparse=True, handle_unknown='ignore')
categorical_converter(raw_data)

In [5]:
from interruptingcow import timeout
dimensions = []
for i in range(42):
    raw_data = data[[col_names[i],'label']]
    features= raw_data.iloc[:,:raw_data.shape[1]]
    labels= raw_data.iloc[:,raw_data.shape[1]-1:]
    labels= labels.values.ravel() # this becomes a 'horizontal' array
    
    #train_test_split
    from sklearn.model_selection import train_test_split
    df= pd.DataFrame(features)
    X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.5, test_size=0.5)
    X_train_features= X_train.iloc[:,:raw_data.shape[1]-1]
    
    #normalization
    from sklearn.preprocessing import StandardScaler

    scaler = StandardScaler().fit(X_train_features)
    #normalization of training dataset 
    X_train.iloc[:,:raw_data.shape[1]-1] = scaler.transform(X_train.iloc[:,:raw_data.shape[1]-1])
    X_train_transformed = pd.DataFrame(X_train)
    X = X_train_transformed[X_train_transformed.label==1]
    X1 = X.iloc[:,:raw_data.shape[1]-1]
    X = X_train_transformed[X_train_transformed.label==0]
    X0 = X.iloc[:,:raw_data.shape[1]-1]
    from sklearn.cluster import KMeans
    try:
        with timeout(3, exception = RuntimeError):
            numberofclasses = 8
            kmeans1 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X1)
            kmeans0 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X0)
    except:
        continue
    label_train = np.append(np.zeros(numberofclasses)+1,np.zeros(numberofclasses))
    data_train = np.append(kmeans1.cluster_centers_,kmeans0.cluster_centers_,axis=0)

    #1 nearest neighbor classification
    from sklearn.neighbors import KNeighborsClassifier 

    knn = KNeighborsClassifier(n_neighbors=1,weights = 'distance')
    knn.fit(data_train, label_train)
    score1=knn.score(X_train.iloc[:,:raw_data.shape[1]-1],X_train.iloc[:,raw_data.shape[1]-1:],sample_weight=None)
    score2=knn.score(X_test.iloc[:,:raw_data.shape[1]-1],X_test.iloc[:,raw_data.shape[1]-1:],sample_weight=None)
    if (score1+score2)/2>0.85:
        print(col_names[i])
        dimensions.append(col_names[i])

service
src_bytes
count
dst_host_same_src_port_rate


Compared to the dimensions reduced by filtering standard divation 'duration','src_bytes','dst_bytes','num_compromised','num_root','count','srv_count','dst_host_count','dst_host_srv_count','label'

In [6]:
dimensions

['service', 'src_bytes', 'count', 'dst_host_same_src_port_rate']

In [7]:
#raw_data = data[['protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'root_shell', 'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'dst_host_count', 'dst_host_srv_count', 'dst_host_same_src_port_rate','label']]
raw_data = data[['service', 'src_bytes', 'count', 'dst_host_same_src_port_rate','label']]
features= raw_data.iloc[:,:raw_data.shape[1]]
labels= raw_data.iloc[:,raw_data.shape[1]-1:]
labels= labels.values.ravel() # this becomes a 'horizontal' array

#train_test_split
from sklearn.model_selection import train_test_split
df= pd.DataFrame(features)
X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.5, test_size=0.5)
X_train_features= X_train.iloc[:,:raw_data.shape[1]-1]

#normalization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train_features)
#normalization of training dataset 
X_train.iloc[:,:raw_data.shape[1]-1] = scaler.transform(X_train.iloc[:,:raw_data.shape[1]-1])
X_train_transformed = pd.DataFrame(X_train)
X = X_train_transformed[X_train_transformed.label==1]
X1 = X.iloc[:,:raw_data.shape[1]-1]
X = X_train_transformed[X_train_transformed.label==0]
X0 = X.iloc[:,:raw_data.shape[1]-1]
from sklearn.cluster import KMeans
numberofclasses = 10
kmeans1 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X1)
kmeans0 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X0)

label_train = np.append(np.zeros(numberofclasses)+1,np.zeros(numberofclasses))
data_train = np.append(kmeans1.cluster_centers_,kmeans0.cluster_centers_,axis=0)

#1 nearest neighbor classification
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=1,weights = 'distance')
knn.fit(data_train, label_train)
score1=knn.score(X_train.iloc[:,:raw_data.shape[1]-1],X_train.iloc[:,raw_data.shape[1]-1:],sample_weight=None)
score2=knn.score(X_test.iloc[:,:raw_data.shape[1]-1],X_test.iloc[:,raw_data.shape[1]-1:],sample_weight=None)

In [8]:
print(score1,score2)

0.9673090158293186 0.8838634716672537
